<a href="https://colab.research.google.com/github/kunalrdeshmukh/RNN_for_Imagecompression/blob/master/Image_compression_using_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

import os
import datetime
import torch
import torchvision
from torch import nn , optim
from torchvision import datasets, transforms
from torch.autograd import Variable, Function
from torch.nn.modules.utils import _pair

!pip install Pillow==4.0.0
!pip install PIL
!pip install image
!pip install lmdb
from PIL import Image

  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [0]:
class Sign(Function):
    """
    Variable Rate Image Compression with Recurrent Neural Networks
    https://arxiv.org/abs/1511.06085
    """

    def __init__(self):
        super(Sign, self).__init__()

    @staticmethod
    def forward(ctx, input, is_training=True):
        # Apply quantization noise while only training
        if is_training:
            prob = input.new(input.size()).uniform_()
            x = input.clone()
            x[(1 - input) / 2 <= prob] = 1
            x[(1 - input) / 2 > prob] = -1
            return x
        else:
            return input.sign()

    @staticmethod
    def backward(ctx, grad_output):
      return grad_output, None

In [0]:
class ConvRNNCellBase(nn.Module):
    def __repr__(self):
        s = (
            '{name}({input_channels}, {hidden_channels}, kernel_size={kernel_size}'
            ', stride={stride}')
        if self.padding != (0, ) * len(self.padding):
            s += ', padding={padding}'
        if self.dilation != (1, ) * len(self.dilation):
            s += ', dilation={dilation}'
        s += ', hidden_kernel_size={hidden_kernel_size}'
        s += ')'
        return s.format(name=self.__class__.__name__, **self.__dict__)

In [0]:
class ConvLSTMCell(ConvRNNCellBase):
    def __init__(self,
                 input_channels,
                 hidden_channels,
                 kernel_size=3,
                 stride=1,
                 padding=0,
                 dilation=1,
                 hidden_kernel_size=1,
                 bias=True):
        super(ConvLSTMCell, self).__init__()
        self.input_channels = input_channels
        self.hidden_channels = hidden_channels

        self.kernel_size = _pair(kernel_size)
        self.stride = _pair(stride)
        self.padding = _pair(padding)
        self.dilation = _pair(dilation)

        self.hidden_kernel_size = _pair(hidden_kernel_size)

        hidden_padding = _pair(hidden_kernel_size // 2)

        gate_channels = 4 * self.hidden_channels
        self.conv_ih = nn.Conv2d(
            in_channels=self.input_channels,
            out_channels=gate_channels,
            kernel_size=self.kernel_size,
            stride=self.stride,
            padding=self.padding,
            dilation=self.dilation,
            bias=bias)

        self.conv_hh = nn.Conv2d(
            in_channels=self.hidden_channels,
            out_channels=gate_channels,
            kernel_size=hidden_kernel_size,
            stride=1,
            padding=hidden_padding,
            dilation=1,
            bias=bias)

        self.reset_parameters()

    def reset_parameters(self):
        self.conv_ih.reset_parameters()
        self.conv_hh.reset_parameters()

    def forward(self, input, hidden):
        hx, cx = hidden
        gates = self.conv_ih(input) + self.conv_hh(hx)

        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = F.sigmoid(ingate)
        forgetgate = F.sigmoid(forgetgate)
        cellgate = F.tanh(cellgate)
        outgate = F.sigmoid(outgate)

        cy = (forgetgate * cx) + (ingate * cellgate)
        hy = outgate * F.tanh(cy)

        return hy, cy

In [5]:
!git clone https://github.com/1zb/pytorch-image-comp-rnn.git

Cloning into 'pytorch-image-comp-rnn'...
remote: Enumerating objects: 114, done.
remote: Total 114 (delta 0), reused 0 (delta 0), pack-reused 114
Receiving objects: 100% (114/114), 2.36 MiB | 30.19 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [6]:
!ls

checkpoint  gdrive  pytorch-image-comp-rnn  sample_data


In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
!ls 

checkpoint  gdrive  pytorch-image-comp-rnn  sample_data


mv: inter-device move failed: 'pytorch-image-comp-rnn' to 'gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn'; unable to remove target: Directory not empty


In [13]:
!python 'gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/train.py' -f 'gdrive/My Drive/Fall 18/297/datasets/Imagenet/train/train/' -N 32 -e 1

total images: 38839; total batches: 1214
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/train.py:170: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  len(train_loader), loss.data[0], bp_t1 - bp_t0, batch_t1 -
[TRAIN] Epoch[1](1/1214); Loss: 0.238569; Backpropagation: 0.7787 sec; Batch: 4.6591 sec
gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/train.py:173: UserWarning: invalid index of a 0-dim tensor. This will be an

In [14]:
!bash 'gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/test/get_kodak.sh'

01
--2018-12-03 00:52:55--  http://r0k.us/graphics/kodak/kodak/kodim01.png
Resolving r0k.us (r0k.us)... 174.127.119.148
Connecting to r0k.us (r0k.us)|174.127.119.148|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 736501 (719K) [image/png]
Saving to: ‘test/images/kodim01.png’

test/images/kodim01 100%[===================>] 719.24K  3.06MB/s    in 0.2s    

2018-12-03 00:52:56 (3.06 MB/s) - ‘test/images/kodim01.png’ saved [736501/736501]

02
--2018-12-03 00:52:56--  http://r0k.us/graphics/kodak/kodak/kodim02.png
Resolving r0k.us (r0k.us)... 174.127.119.148
Connecting to r0k.us (r0k.us)|174.127.119.148|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 617995 (604K) [image/png]
Saving to: ‘test/images/kodim02.png’

test/images/kodim02 100%[===================>] 603.51K  2.70MB/s    in 0.2s    

2018-12-03 00:52:56 (2.70 MB/s) - ‘test/images/kodim02.png’ saved [617995/617995]

03
--2018-12-03 00:52:56--  http://r0k.us/graphics/kodak/kodak/kod

In [18]:
!ls 'checkpoint/'

binarizer_epoch_00000001.pth  decoder_iter_00000000.pth
binarizer_epoch_00000002.pth  encoder_epoch_00000001.pth
binarizer_iter_00000000.pth   encoder_epoch_00000002.pth
decoder_epoch_00000001.pth    encoder_iter_00000000.pth
decoder_epoch_00000002.pth


In [22]:
!python 'gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/encoder.py' --model checkpoint/encoder_epoch_00000002.pth --input test/images/kodim04.png --cuda --output ex --iterations 4

gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/encoder.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  image = Variable(image, volatile=True)
gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/encoder.py:46: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  torch.zeros(batch_size, 256, height // 4, width // 4), volatile=True),
gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/encoder.py:49: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  volatile=True))
gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/encoder.py:51: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  torch.zeros(batch_size, 512, height // 8, width // 8), volatile=True),
gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/encoder.py:54: UserWarning: volati

In [23]:
!python 'gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/encoder.py' --model checkpoint/encoder_epoch_00000002.pth --input test/images/kodim04.png --cuda --output ex --iterations 4

gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/encoder.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  image = Variable(image, volatile=True)
gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/encoder.py:46: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  torch.zeros(batch_size, 256, height // 4, width // 4), volatile=True),
gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/encoder.py:49: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  volatile=True))
gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/encoder.py:51: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  torch.zeros(batch_size, 512, height // 8, width // 8), volatile=True),
gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/encoder.py:54: UserWarning: volati

In [29]:
!python 'gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/decoder.py' --model checkpoint/encoder_epoch_00000001.pth --input ex.npz --cuda --output .

gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/decoder.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  codes = Variable(codes, volatile=True)
Traceback (most recent call last):
  File "gdrive/My Drive/Fall 18/297/Deliverable_4/pytorch-image-comp-rnn/decoder.py", line 35, in <module>
    decoder.load_state_dict(torch.load(args.model))
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 719, in load_state_dict
    self.__class__.__name__, "\n\t".join(error_msgs)))
RuntimeError: Error(s) in loading state_dict for DecoderCell:
	Missing key(s) in state_dict: "conv1.weight", "rnn4.conv_ih.weight", "rnn4.conv_hh.weight", "conv2.weight". 
	Unexpected key(s) in state_dict: "conv.weight". 
	size mismatch for rnn1.conv_ih.weight: copying a param of torch.Size([2048, 512, 3, 3]) from checkpoint, where the shape is torch.Size([1024, 64, 3, 3]) in current model.
	size mismatch for rnn1.conv_hh.weigh